# 0.0 IMPORTS

In [30]:
import pandas as pd

import inflection

import math


## 0.1 Helper Functions

## 0.2 Loading Data

In [19]:

# Lê o arquivo todo de um vez
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)

df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# merge, 1º arquivo de referencia, 2º arquivo anexado a referencia, left, 
# coluna que é igual nos dois datasets e serve como chave para eu fazer o merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')



# 1.0 DESCRIÇÃO DOS DADOS

In [20]:

df1 = df_raw.copy()

## 1.1 Rename Columns

In [22]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
            'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
            'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
            'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map (snakecase, cols_old))

#rename
df1.columns = cols_new


## 1.2 Data Dimensions

In [25]:
#shampe --> printa linhas e colunas de um dataframe 
print(f'Number of Rows: {df1.shape[0]}')
print(f'Number of Cols: {df1.shape[1]}')

Number of Rows: 1017209
Number of Cols: 18


## 1.3 Data Types

In [26]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [27]:
#mundando o tipo da coluna 'date'
df1['date'] = pd.to_datetime(df1['date'])

## 1.4 Check NA

In [29]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5 Fillout NA

In [51]:
# competition_distance 
# 
# tornar os NA em uma distancia maior que a maior distancia já encontrada no dataset
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)         

# competition_open_since_month    
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

# competition_open_since_year     
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

# promo2_since_week               
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year               
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)


# promo_interval

#dicionario que associa o mês ao nome do mês
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Ouc', 11: 'Nov', 12: 'Dec',}
 
#preenche com NA para não ter que comparar usando ISNAN, mas poderia ter feito usando ISNAN
df1['promo_interval'].fillna(0, inplace=True)


#extrai o mês da coluna data e aplica o dicionario para fazer a tradução para o nome do mes
df1['month_map'] = df1['date'].dt.month.map(month_map)

# aplica a condicional para saber se o month_map está dentro daquele intervalo para saber se a loja está ou não em promoção
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [58]:

df1.isna().sum()

df1.sample(5).T


,493560,734597,585849,595410,565867
store,401,598,145,786,233
day_of_week,2,3,3,2,7
date,2014-04-15 00:00:00,2013-09-11 00:00:00,2014-01-22 00:00:00,2014-01-14 00:00:00,2014-02-09 00:00:00
sales,11544,6536,9232,3217,0
customers,922,800,1268,581,0
open,1,1,1,1,0
promo,1,1,1,0,0
state_holiday,0,0,0,0,0
school_holiday,0,1,0,0,0
store_type,a,c,a,a,a


## 1.6 Change Types

In [66]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)




In [ ]:
df1.dtypes

## 1.7 Descriptive Statistical